In [25]:
import ee
import pandas as pd 
import numpy as np
import folium
from folium import plugins

In [26]:
ee.Initialize()
from datetime import datetime, timedelta
from collections import OrderedDict

from calendar import monthrange
def split_years(start_date, end_date):
    """
    Function to split query period to bypass computation timed out (300s query limit of gee)
    Takes input as start and end years
    Gives output as list of start and end periods for each year
    """
  
    start, end = [datetime.strptime(_, "%Y-%m-%d") for _ in [start_date, end_date]]
    start = list(OrderedDict(((start + timedelta(_)).strftime(r"%Y-%m-01"), None) for _ in range((end - start).days)).keys())
    end = [item[:-2]+str(monthrange(int(item[:4]),int(item[5:7]))[1]) for item in start] 
#    print(end)
    return start,end
a,b=split_years("2020-01-01","2020-03-31")
# list(a) 
a

['2020-01-01', '2020-02-01', '2020-03-01']

In [21]:
## Print patch ndvi for mutliplr station_loc stations using loop
import os
os.chdir("D:\\Backup\\Rouhin_Lenovo\\US_project\\Untitled_Folder\\Data\\Volk_merged\\ML_datasets\\Metadata")
st=pd.read_csv("train_test_10.csv")
## Lopping over the locations
def sen1(station_loc,start_date,end_date,dir):
    for i in range(station_loc.shape[0]):
        geometry=ee.Geometry.Point([ station_loc.Lon.iloc[i],station_loc.Lat.iloc[i]]) # Neutral to accomodate Bi1 and Vaira
        grid_list=[]
        s,e=split_years(start_date,end_date)
        for j in range(len(s)):
            bands=["VV","VH","angle"]
            col_no=len(bands)
        # lat=38.382768
        # lon=-121.550491
            col = ee.ImageCollection('COPERNICUS/S1_GRD').filter(ee.Filter.eq('instrumentMode', 'IW')).filterDate(s[j], e[j]) \
                        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
            col=col.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).select(["VV","VH","angle"]).filterBounds(geometry)
            def remove_edge(image):
             edge = image.lt(-30.0)
             maskedImage = image.mask().And(edge.Not())
             return image.updateMask(maskedImage);

            mask_col=col.map(remove_edge)

            ## get the dictionary of values 
            means = mask_col.toBands().reduceRegion(reducer=ee.Reducer.first(),
                                geometry=geometry,
                                scale= 30,
                                crs="EPSG:4326",
                                maxPixels=10e9).getInfo()
            ## Convert to dataframe and readable format
            new_df=pd.DataFrame()
            print(means)
            # df_mean = pd.DataFrame.from_dict(means,orient='index')
            # split_indices = pd.Series([ind.split('_')[-1] for ind in df_mean.index])
            # # forecast_release_hour = pd.Series([ind.rsplit('_')[0].split("F")[0][-2:]  for ind in df_mean.index])
            # # release_date = pd.Series([ind.rsplit('_')[0].split("F")[0][:-2]  for ind in df_mean.index])
            # print(split_indices)
            # cols = pd.Series([ind.rsplit('_')[-1] for ind in df_mean.index])
            # # print(forecast_release_hour.shape)
            # # print(int(len(means)/col_no))
            # # print(split_indices.unique().shape)
            # print(cols,j)
            # # print(pd.DataFrame(df_mean.values.reshape((int(len(means)/col_no), col_no)), columns=cols.unique()))
            # new_df=pd.DataFrame(df_mean.values.reshape((int(len(means)/col_no), col_no)), columns=cols.unique())
            # print(new_df)
            # new_df["Metadata"]=split_indices.unique()

            # new_df["Forecast_release_hour_GMT"]=pd.Series([ind.rsplit('_')[0].split("F")[0][-2:]  for ind in new_df["Metadata"]])
            # new_df["Forecast_release_hour_GMT"]=new_df["Forecast_release_hour_GMT"].astype("int")
            # print(new_df)

            # new_df["Release_date_GMT"]=pd.Series([ind.rsplit('_')[0].split("F")[0][:-2]  for ind in new_df["Metadata"]])
            # new_df["Release_date_GMT"]=pd.to_datetime(new_df["Release_date_GMT"])
            # new_df["Forecast_hour_GMT"]=pd.Series([ind.rsplit('_')[0].split("F")[1]  for ind in new_df["Metadata"]])
            # new_df["Forecast_hour_GMT"]=new_df["Forecast_hour_GMT"].astype("int")
        print(pd.concat(grid_list))
        pd.concat(grid_list).to_csv(dir+station_loc.Name.iloc[i]+".csv")

sen1(st,"2016-10-03","2016-12-03","D:\\Backup\\Rouhin_Lenovo\\US_project\\Untitled_Folder\\Data\\Volk_merged\\ML_datasets\\Sen1\\")

# d = {'Name': ["North", "South"], 'latitude': [33.547044, 33.439129 ],"longitude":[-114.715314, -114.736906]}
# ali_data = pd.DataFrame(data=d)
# ali_data
# ndvi_sen_patch(ali_data,200,"D:\\Backup\\Rouhin_Lenovo\\US_project\\US_Project\\CIMIS\\Analysis_2.0\\")


{'S1A_IW_GRDH_1SDV_20161004T015847_20161004T015921_013334_015430_9992_VH': -16.48964287724834, 'S1A_IW_GRDH_1SDV_20161004T015847_20161004T015921_013334_015430_9992_VV': -11.712408276829658, 'S1A_IW_GRDH_1SDV_20161004T015847_20161004T015921_013334_015430_9992_angle': 37.44950866699219, 'S1A_IW_GRDH_1SDV_20161028T015847_20161028T015912_013684_015F23_9EE6_VH': -12.462611269492776, 'S1A_IW_GRDH_1SDV_20161028T015847_20161028T015912_013684_015F23_9EE6_VV': -7.749885148415057, 'S1A_IW_GRDH_1SDV_20161028T015847_20161028T015912_013684_015F23_9EE6_angle': 37.450592041015625}
{}
{'S1A_IW_GRDH_1SDV_20161215T015846_20161215T015911_014384_017502_79F6_VH': -11.711674462733175, 'S1A_IW_GRDH_1SDV_20161215T015846_20161215T015911_014384_017502_79F6_VV': -10.423969914805866, 'S1A_IW_GRDH_1SDV_20161215T015846_20161215T015911_014384_017502_79F6_angle': 37.44611740112305}


ValueError: No objects to concatenate

In [27]:
## Print patch ndvi for mutliplr station_loc stations using loop
import os
os.chdir("D:\\Backup\\Rouhin_Lenovo\\US_project\\Untitled_Folder\\Data\\Volk_merged\\ML_datasets\\Metadata")
st=pd.read_csv("SEBAL+Volk_all.csv")
## Lopping over the locations
def sen1(station_loc,start_date,end_date,dir):
    for i in range(station_loc.shape[0]):
        geometry=ee.Geometry.Point([ station_loc.Lon.iloc[i],station_loc.Lat.iloc[i]]) # Neutral to accomodate Bi1 and Vaira
        new_df=pd.DataFrame()
        col = ee.ImageCollection("COPERNICUS/S1_GRD").filterDate(start_date, end_date).filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')). \
            filter(ee.Filter.listContains('transmitterReceiverPolarisation', "VV")).filter(ee.Filter.listContains('transmitterReceiverPolarisation', "VH")).filter(ee.Filter.eq('instrumentMode', 'IW'))
        try:
            region = col.getRegion(geometry, int(30),crs="EPSG:4326").getInfo()
        except:
            return pd.DataFrame()
        # stuff the values in a dataframe for convenience      
        df = pd.DataFrame.from_records(region[1:len(region)])
        if df.shape == (0,0):
            return pd.DataFrame()
        else: 
        # use the first list item as column names
            df.columns = region[0]
    
        # drop id column (little value / overhead)
        df.drop('id', axis=1, inplace=True)
    
        # divide the time field by 1000 as in milliseconds
        # while datetime takes seconds to convert unix time
        # to dates
        df.time = df.time / 1000
        df['time'] = pd.to_datetime(df['time'], unit = 's')
        df.rename(columns = {'time': 'date'}, inplace = True)
        df.sort_values(by = 'date')
        print(df)
        df.to_csv(dir+station_loc.Name.iloc[i]+".csv")
    return df
sen1(st,"2014-10-03","2022-12-03","D:\\Backup\\Rouhin_Lenovo\\US_project\\Untitled_Folder\\Data\\Volk_merged\\ML_datasets\\Sen1_all\\")



      longitude   latitude                          date         VV  \
0   -114.515391  32.697374 2015-04-03 01:41:10.349999872 -13.706197   
1   -114.515391  32.697374 2016-03-16 01:41:05.000000000 -15.956374   
2   -114.515391  32.697374 2016-04-21 01:41:11.880000000 -11.104682   
3   -114.515391  32.697374 2016-08-31 01:41:16.000000000 -12.687998   
4   -114.515391  32.697374 2017-01-04 01:40:23.000000000  -7.839398   
..          ...        ...                           ...        ...   
147 -114.515391  32.697374 2021-11-03 01:40:57.000000000  -9.165670   
148 -114.515391  32.697374 2021-11-15 01:40:57.000000000  -7.215319   
149 -114.515391  32.697374 2021-11-27 01:40:57.000000000  -9.356410   
150 -114.515391  32.697374 2021-12-09 01:40:56.000000000  -8.449130   
151 -114.515391  32.697374 2021-12-21 01:40:56.000000000 -13.772971   

            VH      angle  
0   -23.356152  41.882301  
1   -19.384130  41.913517  
2   -16.618227  41.947109  
3   -28.239670  41.950165  
4   -26

,longitude,latitude,date,VV,VH,angle
0,-110.539268,44.953449,2015-03-26 01:11:36.170000128,-8.265143,-12.081759,34.260429
1,-110.539268,44.953449,2015-03-31 01:19:39.789999872,-7.241789,-14.260217,43.920914
2,-110.539268,44.953449,2016-03-08 01:11:36.000000000,-12.487972,-17.740389,34.239632
3,-110.539268,44.953449,2016-03-13 01:19:39.000000000,-15.954489,-20.805177,43.913948
4,-110.539268,44.953449,2016-08-04 01:19:47.410000128,-8.893252,-17.266484,43.910358
...,...,...,...,...,...,...
254,-110.539268,44.953449,2021-11-24 01:19:44.000000000,-10.433772,-16.745210,43.899368
255,-110.539268,44.953449,2021-12-01 01:11:31.000000000,-5.621541,-12.290379,35.316349
256,-110.539268,44.953449,2021-12-06 01:19:44.000000000,-8.070866,-16.826772,43.897549
257,-110.539268,44.953449,2021-12-13 01:11:31.000000000,-6.165056,-8.312412,35.324757
